In [1]:
import requests as r
import pandas as pd
from bs4 import BeautifulSoup as bs
import nltk
from string import punctuation
from collections import Counter
import numpy as np
import re
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
import jieba
from nltk import word_tokenize
import json

In [516]:
data = pd.read_pickle('./forum280.pkl')

In [530]:
stopwords = [line.replace("\n",'') for line in open('stopwords.txt','r').readlines()]

In [531]:
def cutting(text):
    text_c = jieba.cut(text)
    return text_c

In [532]:
def stopwords(w):
    if w not in stop_tokens:
        return w

In [533]:
def regular(w):
    line = re.findall('[\u4e00-\u9fa5]+', w)
    if len(line) > 0:
        return line

In [534]:
syn_dict = {}
with open("syn.txt","r") as f :
    for line in f:
        for word in line.strip("\n").split("\t")[1:]:
            syn_dict[word] = line.strip("\n").split("\t")[0]

In [535]:
def syn(w):
    if w in syn_dict.keys():
        w=syn_dict[w]
        return w
    else:
        return w

In [536]:
#version3 cut first, not what i want
def trimming(data):
    filtered_w = []
    words = cutting(data) 
    for word in words:
        word = regular(word)
        if word is not None:
            stop_w = stopwords(word[0])
            if stop_w is not None:
                syn_w = syn(stop_w)
                filtered_w.append(syn_w)
    return " ".join(filtered_w)

In [537]:
data["tokens"] = data['content'].map(trimming)

Building prefix dict from C:\Users\Cathy\Documents\ETL\myPractice\dict.txt ...
Loading model from cache C:\Users\Cathy\AppData\Local\Temp\jieba.ub90dac2fb499a8444ce8934f4c359f32.cache
Loading model cost 4.176 seconds.
Prefix dict has been built succesfully.


In [538]:
len((data.tokens))

1271

In [2]:
list(data['tokens'])
# 將一行一行的文件變成 "一個" list
# 可以當成英文字的概念就可以用tokenizer 處理

In [540]:
def get_tokens(text):
    words = word_tokenize(text)
    return words

In [541]:
import gensim
sentences = [get_tokens(r) for r in list(data['tokens'])]
model = gensim.models.word2vec.Word2Vec(sentences, min_count=1, size=5)

In [579]:
model.wv.most_similar('悠遊卡')

[('執政黨', 0.9999073147773743),
 ('免費', 0.9998955726623535),
 ('不同意', 0.999894380569458),
 ('說謊', 0.9998858571052551),
 ('走到', 0.9998411536216736),
 ('權益', 0.9998201131820679),
 ('極度', 0.9998135566711426),
 ('更多', 0.9997791051864624),
 ('負擔', 0.9997605085372925),
 ('重', 0.9997496604919434)]

In [3]:
model.wv.most_similar(positive=['中國','經濟' ], negative=['台灣'])

In [573]:
model.most_similar('交流')

C:\Users\Cathy\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('境外', 0.9998736381530762),
 ('就職', 0.9997354745864868),
 ('執照', 0.9994748830795288),
 ('和平', 0.9993702173233032),
 ('商店', 0.9992742538452148),
 ('物', 0.999163031578064),
 ('一貫', 0.9991096258163452),
 ('海運', 0.9990823268890381),
 ('通信', 0.9990628957748413),
 ('新南向', 0.9990330934524536)]